In [1]:
import requests
import pandas as pd
import geojson
import math
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
import itertools
import re
import time

In [2]:
#https://stackoverflow.com/questions/37979644/parse-beautifulsoup-element-into-selenium
def xpath_soup(element):
    """
    Generate xpath of soup element
    :param element: bs4 text or node
    :return: xpath as string
    """
    components = []
    child = element if element.name else element.parent
    for parent in child.parents:
        """
        @type parent: bs4.element.Tag
        """
        previous = itertools.islice(parent.children, 0, parent.contents.index(child))
        xpath_tag = child.name
        xpath_index = sum(1 for i in previous if i.name == xpath_tag) + 1
        components.append(xpath_tag if xpath_index == 1 else '%s[%d]' % (xpath_tag, xpath_index))
        child = parent
    components.reverse()
    return '/%s' % '/'.join(components)

In [34]:
#goes to the coin catalogue and logs in
url = "https://www.doaks.org/resources/coins/catalogue#b_start=0"
browser = webdriver.Chrome(executable_path = '/Users/webbc01/Desktop/ByzCoinsMap/chromedriver')
browser.get(url)
browser.find_element_by_id('__ac_name').send_keys('WebbC01')
browser.find_element_by_id('__ac_password').send_keys('CharlesW')
browser.find_element_by_css_selector('input[type=submit]').click()
#waits for the results of a search to appear
try:
    myElem = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.TextileEntry')))
except TimeoutException:
    print("Loading took too much time!")
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')
#get all options for the mints category
mints = soup.find('select', {'id': 'c11'})
#exclude all mints with 'disabled' attributes
f_mints = []
for mint in mints.children:
    if mint != '\n':
        if len(mint.attrs) < 4:
            f_mints.append(mint)
f_mints = f_mints[:-1]
mints = []
for m in f_mints:
    mints.append(xpath_soup(m))


In [3]:
def get_c_info(html):
    c_info = dict()
    soup = BeautifulSoup(html, 'html.parser')
    imgs = soup.findAll('img', {'alt': 'title'})
    c_info['obverse_url'] = imgs[0]['src']
    c_info['reverse_url'] = imgs[1]['src']
    descriptions = soup.findAll('div', {'class': 'description'})
    c_info['obverse_d'] = descriptions[0].p
    c_info['reverse_d'] = descriptions[1].p
    tbody = soup.find('tbody')
    for c in tbody.children:
        if len(c) > 1:
            c_info[c.th.text] = c.td.text.strip()
    return c_info

def log_in(username, password):
    browser.find_element_by_id('__ac_name').send_keys(password)
    browser.find_element_by_id('__ac_password').send_keys(username)
    browser.find_element_by_css_selector('input[type=submit]').click()
    
def check_by_css(css_selector):
    try:
        myElem = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, css_selector)))
        return True
    except TimeoutException:
        print("Loading took too much time!")
        return False

In [35]:
#try iterating through all records rather than going by mints, because mints don't seem to work
c_info_2 = []
records = []
base = 'https://www.doaks.org/resources/coins/catalogue#b_start='
home = 'https://www.doaks.org/resources/coins/catalogue#b_start=0'
browser.get(home)
for x in range(51):
    browser.get(home)
    check_by_css('div.TextileEntry')
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')
    records = soup.findAll('div', {'class': 'TextileEntrySummary'})
    r_urls = [x.find('a')['href'] for x in records]
    for i, r in enumerate(records):
        check_by_css('div.TextileEntrySummary')
        time.sleep(2)
        r_x = xpath_soup(r)
        try:
            browser.get(r_urls[i])
            time.sleep(3)
            try:
                c_info_2.append(get_c_info(browser.page_source))
                browser.get(home)
            except:
                try:
                    browser.get(r_urls[i].replace('BZC.', 'bzc-'))
                    time.sleep(3)
                    c_info_2.append(get_c_info(browser.page_source))
                    browser.get(home)
                except:
                    browser.get(home)
        except:
            print('couldnt click result:', r)
            browser.get(home)
    home = base + str((x + 1)*15)
    print(home)

WebDriverException: Message: chrome not reachable
  (Session info: chrome=69.0.3497.81)
  (Driver info: chromedriver=2.37.544337 (8c0344a12e552148c185f7d5117db1f28d6c9e85),platform=Mac OS X 10.11.6 x86_64)


In [7]:
len(c_info_2)

718

In [10]:
raw_df = pd.DataFrame(c_info_2)
raw_df['Mint'].unique()

array(['Constantinople', 'Uncertain Mint', 'Thessalonike',
       'Philippopolis', 'Trebizond', 'Yubna', 'Ifriqya', 'Magnesia',
       'Kyzikos', 'Isaur(i)a', 'Seleukeia', 'Alexandria', 'Jerusalem',
       'Syracuse', 'Syrian Mint', 'Ravenna', 'Carthage', 'Fustat',
       'Damascus', 'Cyprus', 'Antioch', 'Nikomedeia', 'Military', 'Spain',
       'Cherson', 'Rome', 'Philadelphia', 'Italian Mint', 'Sis',
       'Selçuk/Theologos', nan, 'Al-Jazira', 'Amida', 'Hisn Kayfa',
       'Merida', 'Ragusa', 'Mytilene', 'Lund', 'Vidin', 'Turnovo', 'Arta',
       'Alexandria ad Issum', 'Neapolis/Nablus', 'Baisan/Skythopolis',
       'Abila?', 'Jerash/Gerasa', 'Halab/Beroea', 'Sicily', 'Catania',
       'Ticinum', 'Beneventum', 'Sardinia', 'Baalbek/Heliopolis',
       'Caesarea'], dtype=object)

In [4]:
raw_df = pd.read_excel('raw_coins.xlsx')

In [5]:
certain_m_r_df = raw_df[raw_df['Mint'] != 'Uncertain Mint']
cmr_df = certain_m_r_df[certain_m_r_df['Mint'].isna() == False]
cmr_df['Date'] = cmr_df['Date'].apply(lambda x: x if type(x) == str else '')
cmr_df['Date'] = [x.replace('circa', '') for x in cmr_df['Date']]
cmr_df['Date'] = [x.replace('?', '') for x in cmr_df['Date']]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [6]:
cmr_dates = cmr_df[cmr_df['Date'] != '']

In [9]:
cmr_dates['Date2'] = cmr_dates['Date'].apply(lambda x: get_dates(str(x)))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
def get_dates(r_date):
    if '/' not in r_date:
        if '–' not in r_date:
            if len(r_date) == 0:
                print('no date')
            else:
                return [r_date]
        else:
            d_list = r_date.split('–')
            return [d_list[0], d_list[1]]
    else:
        if '–' not in r_date:
            d_list = r_date.split('/')
            return [d_list[0], d_list[1]]
        else:
            date_split = r_date.split()
            start , end = 0, 0
            if '/' in date_split[0]:
                start = date_split[0].split('/')[0]
            elif '/' in date_split[-1]:
                end = date_split[-1].split('/')[1]
            if start == 0:
                start = date_split[0]
            if end == 0:
                end = date_split[-1]
            return [start, end]

In [10]:
cmr_dates['Start'] = [int(x[0]) for x in cmr_dates['Date2']]
cmr_dates['End'] = [int(x[-1]) for x in cmr_dates['Date2']]
cmr_dates.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Accession number,Associated Authority,Catalogue ID,Condition,Date,Date of Reign,Denomination,Diameter,Metal,Metal Composition,...,Shape,Undertype,Weight,obverse_d,obverse_url,reverse_d,reverse_url,Date2,Start,End
0,BZC.1973.29,NaN,NaN,NaN,610 – 641,610–641,Decanummium,14.0 mm,Copper,NaN,...,Flat,NaN,2.55 g,"<p>Bust of Herakleios facing, wearing cuirass,...",https://www.doaks.org/resources/coins/catalogu...,"<p><span class=""athenarubyweb"">I</span>. To le...",https://www.doaks.org/resources/coins/catalogu...,"[610 , 641]",610,641
1,BZC.2002.16,NaN,"as DOC 4.1:407, no. 2d.1",NaN,1197 – 1203,1195–1203,Trikephalon (Aspron Trachy),26.0 mm,Electrum,NaN,...,Cup,NaN,4.35 g,"<p>Christ, bearded and nimbate, wearing tunic ...",https://www.doaks.org/resources/coins/catalogu...,<p>Full-length figure of Alexios III Angelos o...,https://www.doaks.org/resources/coins/catalogu...,"[1197 , 1203]",1197,1203
3,BZC.2006.34,NaN,"as DOC 4.1:240, no. 44",NaN,1092 – 1118,NaN,Half Tetarteron,14.0 x 16.0 mm,Copper,NaN,...,Flat,NaN,1.42 g,"<p>Bust of Virgin facing, nimbate, wearing tun...",https://www.doaks.org/resources/coins/catalogu...,"<p>Bust of Alexios I Komnenos facing, wearing ...",https://www.doaks.org/resources/coins/catalogu...,"[1092 , 1118]",1092,1118
4,BZC.2006.38,NaN,"as DOC 4.2:230, no. 31a",NaN,1081 – 1118,1081–1118,Stamenon,22.0 mm,Billon,NaN,...,Flat,NaN,3.37 g,"<p>Christ, bearded and nimbate, wearing tunic ...",https://www.doaks.org/resources/coins/catalogu...,"<p>Alexios I Komnenos standing, wearing diviti...",https://www.doaks.org/resources/coins/catalogu...,"[1081 , 1118]",1081,1118
5,BZC.2009.008,NaN,NaN,NaN,1081 – 1092,1081–1118,Follis,30.0 mm,Copper,NaN,...,Flat,NaN,8.84 g,"<p>Bust of Christ Pantokrator, nimbate with pe...",https://www.doaks.org/resources/coins/catalogu...,<p>Latin cross on base and two steps. In angle...,https://www.doaks.org/resources/coins/catalogu...,"[1081 , 1092]",1081,1092


In [11]:
f_df = cmr_dates.drop(columns=['Date2', 'Shape', 'Undertype', 'Condition'])

In [12]:
mint_coords = pd.read_excel('Mint-Locations.xlsx')
mint_coords = mint_coords.drop(columns = ['Comments'])
mint_coords

,Mint,Longitude,Latitude
0,Abila,35.870040,32.681170
1,Alexandria,29.894378,31.199004
2,Alexandria ad Issum,36.180508,36.579340
3,Al-Jazira,NaN,NaN
4,Amida,37.912245,40.233444
5,Antioch,36.162005,36.200450
6,Arta,NaN,NaN
7,Baalbek/Heliopolis,36.202120,34.011013
8,Baisan/Skythopolis,NaN,NaN
9,Beneventum,10.727138,45.437710


In [13]:
f_df2 = pd.merge(f_df, mint_coords, on="Mint", how="left")


,Accession number,Associated Authority,Catalogue ID,Date,Date of Reign,Denomination,Diameter,Metal,Metal Composition,Mint,...,Ruler,Weight,obverse_d,obverse_url,reverse_d,reverse_url,Start,End,Longitude,Latitude
0,BZC.1973.29,NaN,NaN,610 – 641,610–641,Decanummium,14.0 mm,Copper,NaN,Constantinople,...,Herakleios,2.55 g,"<p>Bust of Herakleios facing, wearing cuirass,...",https://www.doaks.org/resources/coins/catalogu...,"<p><span class=""athenarubyweb"">I</span>. To le...",https://www.doaks.org/resources/coins/catalogu...,610,641,28.961292,41.007791
1,BZC.2002.16,NaN,"as DOC 4.1:407, no. 2d.1",1197 – 1203,1195–1203,Trikephalon (Aspron Trachy),26.0 mm,Electrum,NaN,Constantinople,...,Alexios III Angelos,4.35 g,"<p>Christ, bearded and nimbate, wearing tunic ...",https://www.doaks.org/resources/coins/catalogu...,<p>Full-length figure of Alexios III Angelos o...,https://www.doaks.org/resources/coins/catalogu...,1197,1203,28.961292,41.007791
2,BZC.2006.34,NaN,"as DOC 4.1:240, no. 44",1092 – 1118,NaN,Half Tetarteron,14.0 x 16.0 mm,Copper,NaN,Thessalonike,...,Alexios I Komnenos,1.42 g,"<p>Bust of Virgin facing, nimbate, wearing tun...",https://www.doaks.org/resources/coins/catalogu...,"<p>Bust of Alexios I Komnenos facing, wearing ...",https://www.doaks.org/resources/coins/catalogu...,1092,1118,22.952885,40.628342
3,BZC.2006.38,NaN,"as DOC 4.2:230, no. 31a",1081 – 1118,1081–1118,Stamenon,22.0 mm,Billon,NaN,Philippopolis,...,Alexios I Komnenos,3.37 g,"<p>Christ, bearded and nimbate, wearing tunic ...",https://www.doaks.org/resources/coins/catalogu...,"<p>Alexios I Komnenos standing, wearing diviti...",https://www.doaks.org/resources/coins/catalogu...,1081,1118,24.749474,42.145589
4,BZC.2009.008,NaN,NaN,1081 – 1092,1081–1118,Follis,30.0 mm,Copper,NaN,Trebizond,...,Alexios I Komnenos,8.84 g,"<p>Bust of Christ Pantokrator, nimbate with pe...",https://www.doaks.org/resources/coins/catalogu...,<p>Latin cross on base and two steps. In angle...,https://www.doaks.org/resources/coins/catalogu...,1081,1092,39.723312,41.004269
5,BZC.2006.7.2,NaN,NaN,692 – 697,NaN,Fals,19.0 mm,Copper,NaN,Yubna,...,Abd al-Malik,2.73 g,<p>Standing figure with hand on sword; inscrip...,https://www.doaks.org/resources/coins/catalogu...,"<p><span class=""athenarubyweb""></span>, <em>f...",https://www.doaks.org/resources/coins/catalogu...,692,697,NaN,NaN
6,BZC.2006.7.5,NaN,NaN,692 – 697,NaN,Fals,20.0 mm,Copper,NaN,Yubna,...,Abd al-Malik,4.4 g,"<p>Standing figure with Arab headdress, hand o...",https://www.doaks.org/resources/coins/catalogu...,"<p><span class=""athenarubyweb""></span>, <em>y...",https://www.doaks.org/resources/coins/catalogu...,692,697,NaN,NaN
7,BZC.2006.7.3,NaN,NaN,692 – 697,NaN,Fals,16.0 x 23.0 mm,Copper,NaN,Yubna,...,Abd al-Malik,3.38 g,<p>Standing figure with radiate “halo”; <em>ra...,https://www.doaks.org/resources/coins/catalogu...,"<p><span class=""athenarubyweb""></span>, <em><...",https://www.doaks.org/resources/coins/catalogu...,692,697,NaN,NaN
8,BZC.2006.7.4,NaN,NaN,692 – 697,NaN,Fals,18.0 x 22.0 mm,Copper,NaN,Yubna,...,Abd al-Malik,3.44 g,<p>Standing figure with radiate “halo”; <em>ra...,https://www.doaks.org/resources/coins/catalogu...,"<p><span class=""athenarubyweb""></span><span cl...",https://www.doaks.org/resources/coins/catalogu...,692,697,NaN,NaN
9,BZC.2006.7.1,NaN,NaN,692 – 697,NaN,Fals,19.0 mm,Copper,NaN,Yubna,...,Abd al-Malik,3.18 g,"<p>Standing figure with Arab headdress, hand o...",https://www.doaks.org/resources/coins/catalogu...,"<p><span class=""athenarubyweb""></span>, <em>y...",https://www.doaks.org/resources/coins/catalogu...,692,697,NaN,NaN


In [48]:
f_df2.columns

Index(['Accession number', 'Associated Authority', 'Catalogue ID', 'Date',
       'Date of Reign', 'Denomination', 'Diameter', 'Metal',
       'Metal Composition', 'Mint', 'Relation of Dies', 'Ruler', 'Weight',
       'obverse_d', 'obverse_url', 'reverse_d', 'reverse_url', 'Start', 'End',
       'Longitude', 'Latitude'],
      dtype='object')

In [37]:
mint_coords = mint_coords.dropna(axis=0)
mint_coords

,Mint,Longitude,Latitude
0,Abila,35.870040,32.681170
1,Alexandria,29.894378,31.199004
2,Alexandria ad Issum,36.180508,36.579340
4,Amida,37.912245,40.233444
5,Antioch,36.162005,36.200450
7,Baalbek/Heliopolis,36.202120,34.011013
9,Beneventum,10.727138,45.437710
10,Caesarea,37.252503,35.903735
11,Carthage,10.318000,36.850032
12,Catania,15.088142,37.502959


In [55]:
acc = []
for idx, m in mint_coords.iterrows():
    d_acc = dict()
    temp_p = geojson.Point([m.Longitude, m.Latitude])
    min_date = None
    max_date = None
    for idx, row in f_df2.iterrows():
        r_acc = []
        d = row.to_dict()
        for k in d:
            if type(d[k]) == float:
                r_acc.append(k)
        for r in r_acc:
            del d[r]
        if row.Mint == m.Mint:
            if min_date == None or row.Start < min_date:
                min_date = row.Start
            if max_date == None or row.End > max_date:
                max_date = row.End
            if math.isnan(row.Longitude) == False:
                d_acc[row['Accession number']] = d
    d_acc['Start'] = min_date
    d_acc['End'] = max_date
    min_date = None
    max_date = None
    d_acc['Mint'] = m.Mint
    temp_f = geojson.Feature(geometry=temp_p, properties = d_acc)
    acc.append(temp_f)

In [56]:
f_c = geojson.FeatureCollection(acc)

In [57]:
with open('layer.geojson', 'w') as f:
    geojson.dump(f_c, f)
    f.close()

In [29]:
#updates the mints.geojson file (when I have more )
mint_coords = pd.read_excel('Mint-Locations.xlsx')
acc = []
for idx, row in mint_coords.iterrows():
    if math.isnan(row.Longitude) == False:
        temp_p = geojson.Point([row.Longitude, row.Latitude])
        temp_f = geojson.Feature(geometry=temp_p, properties={'Mint': row.Mint})
        acc.append(temp_f)
        
f_c = geojson.FeatureCollection(acc)

with open('mints.geojson', 'w') as f:
    geojson.dump(f_c, f)
    f.close()

In [ ]:
#function to iterate through mints and through results of each mint
c_info_acc = []
for m in mints:
    browser.find_element_by_xpath(m).click()
    home = browser.current_url
    check_by_css('div.TextileEntry')
    time.sleep(5)
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')
    results = soup.findAll('div', {'class': 'TextileEntry'})
    if len(results) >= 15:
        browser.find_element_by_css_selector('div.listingBar a').click()
        check_by_css('div.TextileEntry')
        m_results = soup.findAll('div', {'class': 'TextileEntry'})
        results = results + m_results
        browser.back()
        time.sleep(3)
    r_urls = [x.find('a')['href'] for x in results]
    for i, r in enumerate(results):
        if i >= 14:
            check_by_css('div.listingBar')
            browser.find_element_by_css_selector('div.listingBar a').click()
            home = browser.current_url
        r_x = xpath_soup(r)
        try:
            browser.find_element_by_xpath(r_x)
            browser.find_element_by_xpath(r_x).click()
        except:
            browser.get(home)
        time.sleep(6)
        try:
            c_info_acc.append(get_c_info(browser.page_source))
            browser.back()
        except:
            try:
                browser.get(r_urls[i].replace('BZC.', 'bzc-'))
                time.sleep(3)
                c_info_acc.append(get_c_info(browser.page_source))
                browser.back()
            except:
                browser.back()
    browser.get(url)
    try:
        myElem = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.TextileEntry')))
    except TimeoutException:
        print("Loading took too much time!")
    
